Related page: https://model.earth/data-commons/docs/data

TO DO: Pull list of timeline data from Google Sheet

TO DO: Reduce state_ids_data object to just state number 01 instead of geoId/01

# Installations and Imports

In [1]:
pip install datacommons_pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import datacommons_pandas as dc
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import requests

# Data Pull for all the states in the USA

In [6]:
stateDict = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California", "CO": "Colorado",
    "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana",
    "ME": "Maine", "MD": "Maryland", "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina", "SD": "South Dakota",
    "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont", "VA": "Virginia", "WA": "Washington",
    "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming",
    "DC": "District of Columbia",
    # US Territories
    "AS": "American Samoa", "GU": "Guam", "MP": "Northern Mariana Islands", "PR": "Puerto Rico", "VI": "Virgin Islands of the U.S."
}

stateData = pd.DataFrame(list(stateDict.items()),columns = ['State','StateName'])
stateData.head()

,State,StateName
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [3]:
def get_state_ids():
  # Getting the states  dcids
  state_dcids = dc.get_places_in(['country/USA'], 'State')
  # Getting the names of the states
  state_names = [dc.get_property_values([name], 'name') for name in state_dcids['country/USA']]

  data = {}
  for entry in state_names:
      for k,v in entry.items():
          data[k] = v[0]

  # Adding Rest 4 US Territories
  for k,v in dc.get_property_values(['geoId/60','geoId/66','geoId/69','geoId/78'], 'name').items():
      data[k] = v[0]
  return data

In [4]:
state_ids_data = get_state_ids()
state_ids_data

{'geoId/01': 'Alabama',
 'geoId/02': 'Alaska',
 'geoId/04': 'Arizona',
 'geoId/05': 'Arkansas',
 'geoId/06': 'California',
 'geoId/08': 'Colorado',
 'geoId/09': 'Connecticut',
 'geoId/10': 'Delaware',
 'geoId/11': 'District of Columbia',
 'geoId/12': 'Florida',
 'geoId/13': 'Georgia',
 'geoId/15': 'Hawaii',
 'geoId/16': 'Idaho',
 'geoId/17': 'Illinois',
 'geoId/18': 'Indiana',
 'geoId/19': 'Iowa',
 'geoId/20': 'Kansas',
 'geoId/21': 'Kentucky',
 'geoId/22': 'Louisiana',
 'geoId/23': 'Maine',
 'geoId/24': 'Maryland',
 'geoId/25': 'Massachusetts',
 'geoId/26': 'Michigan',
 'geoId/27': 'Minnesota',
 'geoId/28': 'Mississippi',
 'geoId/29': 'Missouri',
 'geoId/30': 'Montana',
 'geoId/31': 'Nebraska',
 'geoId/32': 'Nevada',
 'geoId/33': 'New Hampshire',
 'geoId/34': 'New Jersey',
 'geoId/35': 'New Mexico',
 'geoId/36': 'New York',
 'geoId/37': 'North Carolina',
 'geoId/38': 'North Dakota',
 'geoId/39': 'Ohio',
 'geoId/40': 'Oklahoma',
 'geoId/41': 'Oregon',
 'geoId/42': 'Pennsylvania',
 'geo

In [ ]:
# CO2 = Annual_Emissions_CarbonDioxide_NonBiogenic -- State
# Methane = Annual_Emissions_Methane_NonBiogenic -- State
# Population = Count_Person -- State

def fetch_timelines_data():
  # Getting DCIDs for all the states
  state_ids_data = get_state_ids()
  # Getting the timelines data
  timelines_data = {}
  for key,value in state_ids_data.items():
    try:
      timelines_data[value] = {'Population':round(dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']['2022'],2)}
    except:
      timelines_data[value] = {'Population':0.00}
    try:
      timelines_data[value].update({'CO2':round(dc.get_stat_all([key], ['Annual_Emissions_CarbonDioxide_NonBiogenic'])[key]['Annual_Emissions_CarbonDioxide_NonBiogenic']['sourceSeries'][0]['val']['2021'],2)})
    except:
      timelines_data[value].update({'CO2':0.00})
    try:
      timelines_data[value].update({'Methane':round(dc.get_stat_all([key], ['Annual_Emissions_Methane_NonBiogenic'])[key]['Annual_Emissions_Methane_NonBiogenic']['sourceSeries'][0]['val']['2021'],2)})
    except:
      timelines_data[value].update({'Methane':0.00})

  # Creating the dataframe
  normalized_data = []
  for state, values in timelines_data.items():
      entry = {'StateName': state}
      entry.update(values)
      normalized_data.append(entry)
  dataframe = pd.DataFrame(normalized_data)

  # Adding state abbreviations to the dataframe
  dataframe = stateData.merge(dataframe,how = 'inner',on='StateName')

  # Dividing the data by 1000 and rounding to 2 decimals
  # for col in dataframe.iloc[:,2:]:
  #     dataframe[col] = round(dataframe[col]/1000,2)

  return dataframe

In [ ]:
timelines_df = fetch_timelines_data()
timelines_df.head()

,State,StateName,Population,CO2,Methane
0,AL,Alabama,5074296.00,68215710.10,7657440.42
1,AK,Alaska,733583.00,16939106.60,871681.75
2,AZ,Arizona,7359197.00,38392400.30,1361141.25
3,AR,Arkansas,3045637.00,36628789.10,2123889.00
4,CA,California,39029342.00,91754567.10,8737629.75


In [ ]:
# Fetching the data for land area of the states
area_data = gpd.read_file(f'https://www2.census.gov/geo/tiger/GENZ2023/shp/cb_2023_us_state_500k.zip')
area_data["SqMiles"] = round(area_data['ALAND']/ 2589988.110336,2)
area_data.head()

,STATEFP,STATENS,GEOIDFQ,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry,SqMiles
0,35,00897535,0400000US35,35,NM,New Mexico,00,314198587197,726463919,"POLYGON ((-109.05017 31.48000, -109.04984 31.4...",121312.75
1,46,01785534,0400000US46,46,SD,South Dakota,00,196341525171,3387709166,"POLYGON ((-104.05788 44.99761, -104.05078 44.9...",75807.89
2,06,01779778,0400000US06,06,CA,California,00,403673296401,20291770234,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",155859.13
3,21,01779786,0400000US21,21,KY,Kentucky,00,102266598312,2384223544,"MULTIPOLYGON (((-89.40565 36.52817, -89.39869 ...",39485.35
4,01,01779775,0400000US01,01,AL,Alabama,00,131185049346,4582326383,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",50650.83


In [ ]:
timelines_df = timelines_df.merge(area_data[['STUSPS','SqMiles']], how='inner', left_on='State', right_on='STUSPS')
timelines_df.drop(columns = ['STUSPS'],inplace=True)
timelines_df

,State,StateName,Population,CO2,Methane,SqMiles
0,AL,Alabama,5074296.00,68215710.10,7657440.42,50650.83
1,AK,Alaska,733583.00,16939106.60,871681.75,571051.62
2,AZ,Arizona,7359197.00,38392400.30,1361141.25,113655.39
3,AR,Arkansas,3045637.00,36628789.10,2123889.00,51992.70
4,CA,California,39029342.00,91754567.10,8737629.75,155859.13
5,CO,Colorado,5839926.00,50334501.47,8793630.94,103637.06
6,CT,Connecticut,3626205.00,10812606.90,309106.88,4842.40
7,DE,Delaware,1018396.00,5142434.00,401666.25,1948.54
8,FL,Florida,22244823.00,102634265.80,6603958.08,53654.21
9,GA,Georgia,10912876.00,48611013.40,5377826.75,57716.60


In [ ]:
# Converting to a CSV file
timelines_df.to_csv('UN_Timelines_Data.csv', index=False)

## Push CSV to Github


In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from google.colab import userdata

In [ ]:
#user information
token = userdata.get('Githubtoken')
USERNAME = "ultracatx"
REPO_NAME = "data-commons-me"
EMAIL = "ivyzhou752@gmail.com"

repo_url = f"https://{token}@github.com/{USERNAME}/{REPO_NAME}.git"

In [ ]:
#config git global user information
!git config --global user.email {EMAIL}
!git config --global user.name {USERNAME}

In [ ]:
#clone repo to local
!git clone {repo_url}

Cloning into 'data-commons-me'...
remote: Enumerating objects: 645, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 645 (delta 152), reused 169 (delta 85), pack-reused 388 (from 1)
Receiving objects: 100% (645/645), 11.26 MiB | 12.45 MiB/s, done.
Resolving deltas: 100% (228/228), done.


In [ ]:
#move csv from /content to repo
!mv UN_Timelines_Data.csv data-commons-me/docs/data

In [ ]:
%cd /content/data-commons-me/docs/data

/content/data-commons-me/docs/data


In [ ]:
!git add UN_Timelines_Data.csv

In [ ]:
!git commit -m "push csv to repo"

[main d738a91] test
 1 file changed, 57 deletions(-)
 delete mode 100644 docs/data/UN_Timelines_Data.csv


In [ ]:
!git push {repo_url}

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 383 bytes | 383.00 KiB/s, done.
Total 4 (delta 2), reused 1 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   https://github.com/Ultracatx/data-commons-me.git
To https://github.com/ultracatx/data-commons-me.git
   a66375a..d738a91  main -> main


# Data Pull for all the countries

In [ ]:
def get_country_ids():
    country_dcids = dc.get_places_in(['Earth'], 'Country')
    country_names = {dcid: dc.get_property_values([dcid], 'name')[dcid][0] for dcid in country_dcids['Earth']}
    return country_names

country_ids_data = get_country_ids()
country_ids_data

{'country/ABW': 'Aruba',
 'country/AFG': 'Afghanistan',
 'country/AGO': 'Angola',
 'country/AIA': 'Anguilla',
 'country/ALA': 'Åland Islands',
 'country/ALB': 'Albania',
 'country/AND': 'Andorra',
 'country/ANT': 'Netherlands Antilles',
 'country/ARE': 'United Arab Emirates',
 'country/ARG': 'Argentina',
 'country/ARM': 'Armenia',
 'country/ASM': 'American Samoa',
 'country/ATA': 'Antarctica',
 'country/ATB': 'British Antarctic Territory',
 'country/ATF': 'French Southern Territories',
 'country/ATG': 'Antigua and Barbuda',
 'country/ATN': 'Dronning Maud Land',
 'country/AUS': 'Australia',
 'country/AUT': 'Austria',
 'country/AZE': 'Azerbaijan',
 'country/BDI': 'Burundi',
 'country/BEL': 'Belgium',
 'country/BEN': 'Benin',
 'country/BES': 'Bonaire, Sint Eustatius and Saba',
 'country/BFA': 'Burkina Faso',
 'country/BGD': 'Bangladesh',
 'country/BGR': 'Bulgaria',
 'country/BHR': 'Bahrain',
 'country/BHS': 'Bahamas',
 'country/BIH': 'Bosnia and Herzegovina',
 'country/BLM': 'Saint Barthé

In [ ]:
!pip install pycountry
import pycountry
def create_country_dataframe():
    country_dict = {country.alpha_2: country.name for country in pycountry.countries}
    country_data = pd.DataFrame(list(country_dict.items()), columns=['CountryCode', 'CountryName'])

    return country_data

country_data = create_country_dataframe()
country_data

,CountryCode,CountryName
0,AW,Aruba
1,AF,Afghanistan
2,AO,Angola
3,AI,Anguilla
4,AX,Åland Islands
...,...,...
244,WS,Samoa
245,YE,Yemen
246,ZA,South Africa
247,ZM,Zambia


In [ ]:
# CO2 = Annual_Emissions_CarbonDioxide_NonBiogenic
# Methane = Annual_Emissions_Methane_NonBiogenic
# Population = Count_Person


def fetch_timelines_data_country():
    # Getting DCIDs for all the countries
    country_ids_data = get_country_ids()

    # Getting the timelines data
    timelines_data = {}
    for key, value in country_ids_data.items():
        try:
            timelines_data[value] = {'Population': round(dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']['2022'], 2)}
        except:
            timelines_data[value] = {'Population': 0.00}
        try:
            timelines_data[value].update({'CO2': round(dc.get_stat_all([key], ['Annual_Emissions_CarbonDioxide_NonBiogenic'])[key]['Annual_Emissions_CarbonDioxide_NonBiogenic']['sourceSeries'][0]['val']['2021'], 2)})
        except:
            timelines_data[value].update({'CO2': 0.00})
        try:
            timelines_data[value].update({'Methane': round(dc.get_stat_all([key], ['Annual_Emissions_Methane_NonBiogenic'])[key]['Annual_Emissions_Methane_NonBiogenic']['sourceSeries'][0]['val']['2021'], 2)})
        except:
            timelines_data[value]. update({'Methane': 0.00})

    # Creating the dataframe
    normalized_data = []
    for country, values in timelines_data.items():
        entry = {'CountryName': country}
        entry.update(values)
        normalized_data.append(entry)
    dataframe = pd.DataFrame(normalized_data)

    return dataframe

timelines_country = fetch_timelines_data_country()
timelines_country.head()

,CountryName,Population,CO2,Methane
0,Aruba,106445.00,0.00,0.00
1,Afghanistan,41128771.00,0.00,0.00
2,Angola,35588987.00,0.00,0.00
3,Anguilla,0.00,0.00,0.00
4,Åland Islands,0.00,0.00,0.00


combined

In [ ]:
# Country
dc.get_stat_all(['country/USA'], ['Annual_Amount_Emissions_CarbonDioxide'])['country/USA']['Annual_Amount_Emissions_CarbonDioxide']

{'sourceSeries': [{'val': {'1966': 3547095488,
    '2005': 5703154482,
    '1967': 3684656198,
    '1984': 4485577869,
    '1985': 4514313221,
    '2012': 4902939710,
    '2001': 5702105773,
    '1986': 4478048562,
    '1980': 4595789109,
    '2020': 4257738096,
    '2010': 5352049740,
    '1969': 4038171997,
    '1993': 4952699132,
    '2008': 5512449220,
    '2003': 5610687172,
    '1994': 5024064992,
    '1960': 2813809267,
    '2018': 4909996024,
    '2021': 4483810505,
    '1975': 4355839181,
    '2002': 5545413879,
    '2007': 5686667534,
    '1989': 4904547626,
    '1968': 3898241195,
    '2009': 5120626218,
    '2017': 4761301727,
    '1977': 4781620648,
    '1998': 5545388048,
    '2013': 5038524675,
    '1974': 4539981890,
    '1961': 2832097916,
    '1970': 4231974784,
    '2011': 5128105384,
    '2019': 4744449220,
    '1963': 3085190502,
    '1999': 5563819024,
    '1971': 4289035482,
    '1991': 4765689535,
    '2016': 4838476243,
    '2015': 4928610664,
    '1976': 46168

# Code for Priyanka

In [5]:
def get_county_ids():
  # Getting the county  dcids
  county_dcids = {}
  county_names = []
  for key,value in state_ids_data.items():
    county_dcids.update(dc.get_places_in([key], 'County'))
  # Getting the names of the county
    county_names = {}
  for key,value in county_dcids.items():
    for county_id in value:
      county_names[county_id] = dc.get_property_values([county_id], 'name')[county_id][0]
  return county_names

In [6]:
county_ids_data = get_county_ids()
county_ids_data

{'geoId/01001': 'Autauga County',
 'geoId/01003': 'Baldwin County',
 'geoId/01005': 'Barbour County',
 'geoId/01007': 'Bibb County',
 'geoId/01009': 'Blount County',
 'geoId/01011': 'Bullock County',
 'geoId/01013': 'Butler County',
 'geoId/01015': 'Calhoun County',
 'geoId/01017': 'Chambers County',
 'geoId/01019': 'Cherokee County',
 'geoId/01021': 'Chilton County',
 'geoId/01023': 'Choctaw County',
 'geoId/01025': 'Clarke County',
 'geoId/01027': 'Clay County',
 'geoId/01029': 'Cleburne County',
 'geoId/01031': 'Coffee County',
 'geoId/01033': 'Colbert County',
 'geoId/01035': 'Conecuh County',
 'geoId/01037': 'Coosa County',
 'geoId/01039': 'Covington County',
 'geoId/01041': 'Crenshaw County',
 'geoId/01043': 'Cullman County',
 'geoId/01045': 'Dale County',
 'geoId/01047': 'Dallas County',
 'geoId/01049': 'DeKalb County',
 'geoId/01051': 'Elmore County',
 'geoId/01053': 'Escambia County',
 'geoId/01055': 'Etowah County',
 'geoId/01057': 'Fayette County',
 'geoId/01059': 'Franklin 

#Code Updated by Priyanka - Population Data Pull for all Counties

In [11]:
dc.get_stat_all(['geoId/01001'], ['Count_Person'])
#['geoId/01001']['Count_Person']['sourceSeries'][0]['val']


{'geoId/01001': {'Count_Person': {'sourceSeries': [{'val': {'2002': 45909,
      '1992': 35018,
      '1990': 34222,
      '2009': 54135,
      '2007': 52405,
      '1980': 32259,
      '1982': 32036,
      '2006': 51328,
      '1988': 33636,
      '1983': 32054,
      '2003': 46800,
      '1987': 33268,
      '1985': 32245,
      '1971': 25500,
      '2020': 56145,
      '2005': 49676,
      '2001': 44889,
      '1996': 39154,
      '2004': 48366,
      '1973': 28500,
      '1997': 40251,
      '1991': 34356,
      '2014': 54922,
      '2012': 54970,
      '1984': 32134,
      '2018': 55533,
      '1975': 29700,
      '2021': 59095,
      '2013': 54747,
      '1998': 41293,
      '2016': 55302,
      '1986': 32893,
      '1989': 33996,
      '1970': 24460,
      '1995': 38218,
      '1978': 30900,
      '2011': 55229,
      '1972': 27200,
      '1993': 36002,
      '2010': 54761,
      '2015': 54903,
      '2022': 59759,
      '1977': 30500,
      '1979': 32100,
      '2019': 55769,
 

In [32]:
d=dc.get_stat_all(['geoId/01001'], ['Count_Person'])['geoId/01001']['Count_Person']['sourceSeries'][0]['val']
#['geoId/01001']['Count_Person']['sourceSeries'][0]['val']
sorted(d)



['1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [27]:
population_data_all_county = {}
for key,value in county_ids_data.items():
    try:
        #population_data_all_county[value] = {str(1970): dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']['1970']}
        stats = dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']
        population_data_all_county[value] = {str(year): stats.get(str(year), 0) for year in range(1970, 2023)}
    except:
        population_data_all_county[value] = {str(1970): 0}
        population_data_all_county[value] = {str(year): 0 for year in range(1970, 2023)}
    for year in range(1971,2023):
        try:
            population_data_all_county[value] = {str(year): stats.get(str(year), 0) for year in range(1970, 2023)}
            #population_data_all_county[value].update({str(year): dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val'][str(year)]})
        except:
            #population_data_all_county[value].update({str(year): 0})
            population_data_all_county[value] = {str(year): 0 for year in range(1970, 2023)}

# for key, value in county_ids_data.items():
#     try:
#         stats = dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']
#         population_data_all_county[value] = {str(year): stats.get(str(year), 0) for year in range(1970, 2023)}
#     except:
#         population_data_all_county[value] = {str(year): 0 for year in range(1970, 2023)}

In [28]:
population_data_all_county

{'Autauga County': {'1970': 24460,
  '1971': 25500,
  '1972': 27200,
  '1973': 28500,
  '1974': 29300,
  '1975': 29700,
  '1976': 29900,
  '1977': 30500,
  '1978': 30900,
  '1979': 32100,
  '1980': 32259,
  '1981': 31985,
  '1982': 32036,
  '1983': 32054,
  '1984': 32134,
  '1985': 32245,
  '1986': 32893,
  '1987': 33268,
  '1988': 33636,
  '1989': 33996,
  '1990': 34222,
  '1991': 34356,
  '1992': 35018,
  '1993': 36002,
  '1994': 36976,
  '1995': 38218,
  '1996': 39154,
  '1997': 40251,
  '1998': 41293,
  '1999': 42193,
  '2000': 44021,
  '2001': 44889,
  '2002': 45909,
  '2003': 46800,
  '2004': 48366,
  '2005': 49676,
  '2006': 51328,
  '2007': 52405,
  '2008': 53277,
  '2009': 54135,
  '2010': 54761,
  '2011': 55229,
  '2012': 54970,
  '2013': 54747,
  '2014': 54922,
  '2015': 54903,
  '2016': 55302,
  '2017': 55448,
  '2018': 55533,
  '2019': 55769,
  '2020': 56145,
  '2021': 59095,
  '2022': 59759},
 'Baldwin County': {'1970': 34240,
  '1971': 34900,
  '1972': 33900,
  '1973': 3

In [36]:
def County_Data_to_Dataframe(data):
  # Adding County as a Column and Converting the data to dataframe
  normalized_data = []
  for county, values in data.items():
      entry = {'CountyName': county}
      entry.update(values)
      normalized_data.append(entry)
  dataframe = pd.DataFrame(normalized_data)


  return dataframe


In [37]:
population_dataframe_all_county = County_Data_to_Dataframe(population_data_all_county)
population_dataframe_all_county.head()

,CountyName,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Autauga County,24460,25500,27200,28500,29300,29700,29900,30500,30900,...,54747,54922,54903,55302,55448,55533,55769,56145,59095,59759
1,Baldwin County,34240,34900,33900,34200,34300,35700,34600,34800,33800,...,46191,45920,45566,45260,44993,45001,45006,45099,43781,43635
2,Barbour County,14030,14700,15000,15100,15100,15400,15900,16000,16000,...,16875,16909,16980,16752,16530,16524,16466,16444,15468,15414
3,Bibb County,143366,145800,147200,147600,147500,149100,148800,150700,151300,...,155138,154447,154068,153212,152920,153069,153191,152737,156762,156197
4,Blount County,63744,64200,64400,66900,68400,68900,70300,72100,73600,...,124937,125932,127047,128356,130106,131595,133396,134751,137605,139958


In [38]:
population_dataframe_all_county

,CountyName,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Autauga County,24460,25500,27200,28500,29300,29700,29900,30500,30900,...,54747,54922,54903,55302,55448,55533,55769,56145,59095,59759
1,Baldwin County,34240,34900,33900,34200,34300,35700,34600,34800,33800,...,46191,45920,45566,45260,44993,45001,45006,45099,43781,43635
2,Barbour County,14030,14700,15000,15100,15100,15400,15900,16000,16000,...,16875,16909,16980,16752,16530,16524,16466,16444,15468,15414
3,Bibb County,143366,145800,147200,147600,147500,149100,148800,150700,151300,...,155138,154447,154068,153212,152920,153069,153191,152737,156762,156197
4,Blount County,63744,64200,64400,66900,68400,68900,70300,72100,73600,...,124937,125932,127047,128356,130106,131595,133396,134751,137605,139958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,Vega Baja,0,0,0,0,0,0,0,0,0,...,58782,57915,56858,55866,54754,53371,52192,51089,54544,54182
1963,Vieques,0,0,0,0,0,0,0,0,0,...,9273,9217,9130,9046,8931,8771,8642,8508,8317,8199
1964,Villalba,0,0,0,0,0,0,0,0,0,...,25618,25196,24685,24186,23659,22993,22403,21899,22341,21984
1965,Yabucoa,0,0,0,0,0,0,0,0,0,...,37404,36903,36279,35670,35025,34149,33499,32867,31047,30313


In [39]:
population_dataframe_all_county.to_csv('County_data_all_years.csv',index=False)


In [40]:
pd.read_csv('County_data_all_years.csv')

,CountyName,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Autauga County,24460,25500,27200,28500,29300,29700,29900,30500,30900,...,54747,54922,54903,55302,55448,55533,55769,56145,59095,59759
1,Baldwin County,34240,34900,33900,34200,34300,35700,34600,34800,33800,...,46191,45920,45566,45260,44993,45001,45006,45099,43781,43635
2,Barbour County,14030,14700,15000,15100,15100,15400,15900,16000,16000,...,16875,16909,16980,16752,16530,16524,16466,16444,15468,15414
3,Bibb County,143366,145800,147200,147600,147500,149100,148800,150700,151300,...,155138,154447,154068,153212,152920,153069,153191,152737,156762,156197
4,Blount County,63744,64200,64400,66900,68400,68900,70300,72100,73600,...,124937,125932,127047,128356,130106,131595,133396,134751,137605,139958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,Vega Baja,0,0,0,0,0,0,0,0,0,...,58782,57915,56858,55866,54754,53371,52192,51089,54544,54182
1963,Vieques,0,0,0,0,0,0,0,0,0,...,9273,9217,9130,9046,8931,8771,8642,8508,8317,8199
1964,Villalba,0,0,0,0,0,0,0,0,0,...,25618,25196,24685,24186,23659,22993,22403,21899,22341,21984
1965,Yabucoa,0,0,0,0,0,0,0,0,0,...,37404,36903,36279,35670,35025,34149,33499,32867,31047,30313


In [9]:
# # CO2 = Annual_Emissions_CarbonDioxide_NonBiogenic -- County
# # Methane = Annual_Emissions_Methane_NonBiogenic -- County
# # Population = Count_Person -- County

def fetch_emmision_data_county():
  # Getting DCIDs for all the states
  county_ids_data = get_county_ids()
  # Getting the timelines data
  timelines_data = {}
  for key,value in county_ids_data.items():
    try:
      timelines_data[value] = {'Population':round(dc.get_stat_all([key], ['Count_Person'])[key]['Count_Person']['sourceSeries'][0]['val']['2022'],2)}
    except:
      timelines_data[value] = {'Population':0.00}
    try:
      timelines_data[value].update({'CO2':round(dc.get_stat_all([key], ['Annual_Emissions_CarbonDioxide_NonBiogenic'])[key]['Annual_Emissions_CarbonDioxide_NonBiogenic']['sourceSeries'][0]['val']['2021'],2)})
    except:
      timelines_data[value].update({'CO2':0.00})
    try:
      timelines_data[value].update({'Methane':round(dc.get_stat_all([key], ['Annual_Emissions_Methane_NonBiogenic'])[key]['Annual_Emissions_Methane_NonBiogenic']['sourceSeries'][0]['val']['2021'],2)})
    except:
      timelines_data[value].update({'Methane':0.00})

  # Creating the dataframe
  normalized_data = []
  for county, values in timelines_data.items():
      entry = {'CountyName': county}
      entry.update(values)
      normalized_data.append(entry)
  dataframe = pd.DataFrame(normalized_data)

#   # Adding state abbreviations to the dataframe
#   #dataframe = stateData.merge(dataframe,how = 'inner',on='StateName')

  # Dividing the data by 1000 and rounding to 2 decimals
  # for col in dataframe.iloc[:,2:]:
  #     dataframe[col] = round(dataframe[col]/1000,2)

  return dataframe

In [10]:

timelines_county = fetch_emmision_data_county()
timelines_county.head()

,CountyName,Population,CO2,Methane
0,Autauga County,59759.00,4635470.10,65884.00
1,Baldwin County,43635.00,11190.00,6671.75
2,Barbour County,15414.00,0.00,612664.00
3,Bibb County,156197.00,233019.70,75208.50
4,Blount County,139958.00,203575.70,84954.25


Optimized Code for Co2 and Methane emmisiion data for conty

In [11]:
import concurrent.futures


def fetch_timeline_data_for_county(county_key):
    """Fetch population, CO2, and methane data for a single county."""
    county_data = {'Population': 0.00, 'CO2': 0.00, 'Methane': 0.00}

    try:
        population_data = dc.get_stat_all([county_key], ['Count_Person'])
        county_data['Population'] = round(population_data[county_key]['Count_Person']['sourceSeries'][0]['val']['2022'], 2)
    except KeyError:
        pass  # Population data remains 0.00 if there's an error

    try:
        co2_data = dc.get_stat_all([county_key], ['Annual_Emissions_CarbonDioxide_NonBiogenic'])
        county_data['CO2'] = round(co2_data[county_key]['Annual_Emissions_CarbonDioxide_NonBiogenic']['sourceSeries'][0]['val']['2021'], 2)
    except KeyError:
        pass  # CO2 data remains 0.00 if there's an error

    try:
        methane_data = dc.get_stat_all([county_key], ['Annual_Emissions_Methane_NonBiogenic'])
        county_data['Methane'] = round(methane_data[county_key]['Annual_Emissions_Methane_NonBiogenic']['sourceSeries'][0]['val']['2021'], 2)
    except KeyError:
        pass  # Methane data remains 0.00 if there's an error

    return county_data

def fetch_timelines_data_county():
    # Getting DCIDs for all the counties
    county_ids_data = get_county_ids()

    # Using ThreadPoolExecutor for parallel data fetching
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_county = {executor.submit(fetch_timeline_data_for_county, key): value for key, value in county_ids_data.items()}

        timelines_data = {}
        for future in concurrent.futures.as_completed(future_to_county):
            county_name = future_to_county[future]
            try:
                timelines_data[county_name] = future.result()
            except Exception as e:
                timelines_data[county_name] = {'Population': 0.00, 'CO2': 0.00, 'Methane': 0.00}
                print(f"Error fetching data for {county_name}: {e}")

    # Creating the dataframe
    normalized_data = [{'CountyName': county, **values} for county, values in timelines_data.items()]
    dataframe = pd.DataFrame(normalized_data)

    return dataframe


In [12]:
timelines_county_df = fetch_timelines_data_county()
timelines_county_df.head()

Error fetching data for Miami County: [WinError 10054] An existing connection was forcibly closed by the remote host


,CountyName,Population,CO2,Methane
0,Autauga County,59759.00,4635470.10,65884.00
1,Calhoun County,6068.00,0.00,0.00
2,Clay County,7814.00,0.00,0.00
3,Bibb County,156197.00,233019.70,75208.50
4,Choctaw County,14358.00,1008684.20,2984.75


In [13]:
timelines_county_df

,CountyName,Population,CO2,Methane
0,Autauga County,59759.00,4635470.10,65884.00
1,Calhoun County,6068.00,0.00,0.00
2,Clay County,7814.00,0.00,0.00
3,Bibb County,156197.00,233019.70,75208.50
4,Choctaw County,14358.00,1008684.20,2984.75
...,...,...,...,...
1962,Utuado,28155.00,0.00,0.00
1963,Vega Baja,54182.00,0.00,0.00
1964,Villalba,21984.00,0.00,0.00
1965,Yabucoa,30313.00,0.00,0.00


In [15]:
timelines_county_df.to_csv('County_Co2_Methane_data.csv',index=False)
pd.read_csv('County_Co2_Methane_data.csv')

,CountyName,Population,CO2,Methane
0,Autauga County,59759.00,4635470.10,65884.00
1,Calhoun County,6068.00,0.00,0.00
2,Clay County,7814.00,0.00,0.00
3,Bibb County,156197.00,233019.70,75208.50
4,Choctaw County,14358.00,1008684.20,2984.75
...,...,...,...,...
1962,Utuado,28155.00,0.00,0.00
1963,Vega Baja,54182.00,0.00,0.00
1964,Villalba,21984.00,0.00,0.00
1965,Yabucoa,30313.00,0.00,0.00
